## OpenCV I/O functionality

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

The above piece of code imported all the packages we need through this tutorial.

Now, let's load an image.

### Load, display and save images
Use the function [`cv2.imread()`](http://docs.opencv.org/3.0-beta/modules/imgcodecs/doc/reading_and_writing_images.html?highlight=imread#cv2.imread) to read an image. The function accepts two arguments, the first argument is the path of the image and the second argument is a flag indicating how to decode the image, there are three options:
- `cv2.IMREAD_COLOR`: load the image in color mode
- `cv2.IMREAD_GRAYSCALE`: load the image in grayscale mode
- `cv2.IMREAD_UNCHANGED`: load the image in color+alpha mode if the image has an alpha (controls transparency) channel, otherwise in color mode

Use the function [`cv2.imshow()`](http://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html?highlight=imshow#cv2.imshow) to display an image. This function accepts two arguments, the first argument is the window name and the second one is the image, which is an numpy array.

In [ ]:
# load the image
image = cv2.imread('dog.png', cv2.IMREAD_COLOR)
# TODO: check size of images loaded with different options
print image.shape

In [ ]:
# display the image
cv2.imshow('display', image)

Running the code above doesn't show us the image, we need to run the "event loop" and go into the display thread by calling [`cv2.waitKey()`](http://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html?highlight=waitkey#waitkey). You can either give this function an integer argument, which is the amount of time to wait (in milliseconds) or leave out the argument, which means the display thread will wait forever until you press a key. The value returned is the ASCII code of the key you pressed. We can use this to wait for a specific key stroke and manipulate the image in different ways according to the key pressed.

In [ ]:
# press any key to close the window
cv2.waitKey()
# close the window
cv2.destroyWindow('display')

**Example**

Wait for a specific key, say 'x'.

In [ ]:
image = cv2.imread('dog.png', cv2.IMREAD_UNCHANGED)
key = None
while key != ord('x'):
    cv2.imshow('display', image)
    # wait for 30 ms
    key = cv2.waitKey(30)
# close the window
cv2.destroyWindow('display')

**Example** 

Do different things to the image according to the key pressed.
- 'v': flip the image vertically
- 'h': flip the image horizontally
- 'u': upsample the image by a factor of 2
- 'd': downsample the image by a factor of 2
- 'r': show the red channel of the image
- 'g': show the green channel of the image
- 'b': show the blue channel of the image
- 's': save the modified image

In [ ]:
image = cv2.imread('dog.png', cv2.IMREAD_UNCHANGED)
key = None
while key != ord('x'):
    cv2.imshow('original', image)
    if key == ord('v'):
        image_new = np.flipud(image)
        cv2.imshow('modified', image_new)
    elif key == ord('h'):
        image_new = np.fliplr(image)
        cv2.imshow('modified', image_new)
    elif key == ord('u'):
        # resize the input image to desired size
        # 1st arg: image
        # 2nd arg: desired size
        # 3rd arg: interpolation method, default is bilinear
        image_new = cv2.resize(image, 
                           dsize=(1024, 1024), 
                           interpolation=cv2.INTER_LINEAR)
        cv2.imshow('modified', image_new)
    elif key == ord('d'):
        image_new = cv2.resize(image, 
                           (256, 256),
                          interpolation=cv2.INTER_LINEAR)
        cv2.imshow('modified', image_new)   
    elif key == ord('r'):
        image_new = image.copy()
        image_new[..., [0,1]] = 0
        cv2.imshow('modified', image_new)
    elif key == ord('g'):
        image_new = image.copy()
        image_new[..., [0,2]] = 0
        cv2.imshow('modified', image_new)
    elif key == ord('b'):
        image_new = image.copy()
        image_new[..., [1,2]] = 0
        cv2.imshow('modified', image_new)
    elif key == ord('s'):
        # save the image to a given path
        # 1st arg: desired image path
        # 2nd arg: image data
        cv2.imwrite('dog_new.png', image_new)                
    # wait for 30 ms
    key = cv2.waitKey(30)
# close all window
cv2.destroyAllWindows()

**useful functions**
- [`cv2.imread()`](http://docs.opencv.org/3.0-beta/modules/imgcodecs/doc/reading_and_writing_images.html?highlight=imread#cv2.imread)
- [`cv2.imshow()`](http://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html?highlight=imshow#cv2.imshow)
- [`cv2.imwrite()`](http://docs.opencv.org/3.0-beta/modules/imgcodecs/doc/reading_and_writing_images.html?highlight=imwrite#cv2.imwrite)
- [`cv2.waitKey()`](http://docs.opencv.org/3.0-beta/modules/highgui/doc/user_interface.html?highlight=waitkey#waitkey)
- [`cv2.resize()`](http://docs.opencv.org/3.0-beta/modules/imgproc/doc/geometric_transformations.html?highlight=resize#cv2.resize)
- More on [reading & writing images](http://docs.opencv.org/3.0-beta/modules/imgcodecs/doc/imgcodecs.html)
- More on [geometric image transformations]( http://docs.opencv.org/3.0-beta/modules/imgproc/doc/geometric_transformations.html)

### Use webcam to capture a video

#### launch the camera

In [ ]:
# Create an object which can capture images.
cap = cv2.VideoCapture()
# Camera devices are indexed by integers, 
# typically your laptop webcam has device number 0.
# Do a small loop to find the proper camera and open it.
for i in range(10):
    if cap.open(i):
        print 'camera {} launched'.format(i)
        break
        

#### grab images from the camera

In [ ]:
key = None
while key != ord('x'):
    status, image = cap.read()
    assert status, 'failed to grab image from camera'
    cv2.imshow('display', image)
    key = cv2.waitKey(30)
cv2.destroyWindow('display')

#### release the camera

In [ ]:
# release the camera
cap.release()

### Application: face detection in a video

TODO: theory behind face detection ... and then the related functions.

In [ ]:
# Create a cascade face detector.
configure_file = 'haarcascade_frontalface_default.xml'
face_detector = cv2.CascadeClassifier(configure_file)
# let's load the lena image
image = cv2.imread('lena.jpg')
# The face detector takes a gray scale image as input,
# so we need to convert the color image to grayscale image first.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_detector.detectMultiScale(gray, 1.1, 2)
for (x, y, w, h) in faces:
    image = cv2.rectangle(image, (x,y), (x+w, y+h), (255, 0, 0), 2)
cv2.imshow('display', image)
cv2.waitKey()
cv2.destroyWindow('display')

#### put everything together

In [ ]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Create an object which can capture images.
cap = cv2.VideoCapture()
# Camera devices are indexed by integers, 
# typically your laptop webcam has device number 0.
# Do a small loop to find the proper camera and open it.
for i in range(10):
    if cap.open(i):
        print 'camera {} launched'.format(i)
        break
key = None
while key != ord('x'):
    status, image = cap.read()
    assert status, 'failed to grab image from camera'
    # convert color image to grayscale image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    
    faces = face_detector.detectMultiScale(gray, 1.1, 2)
    for (x,y,w,h) in faces:
        image = cv2.rectangle(image, (x,y), (x+w,y+h), (255,0,0), 2)               
    cv2.imshow('display', image)
    key = cv2.waitKey(30)
cv2.destroyWindow('display')
# release the camera
cap.release()

In [ ]:
cap.release()

In [ ]:
face_cascade

## Useful resources
- [OpenCV API reference](http://docs.opencv.org/2.4/modules/refman.html)
- [OpenCV official repository](https://github.com/opencv/opencv)
- [OpenCV code samples in Python](https://github.com/opencv/opencv/tree/master/samples/python)
- [vlfeat tutorials](http://www.vlfeat.org/overview/tut.html)
- [numpy tutorial](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html)
- [Jupyter notebook](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/)
